# 외부효과와 입출력 - A

## 1. 효과의 추출 : **"모든 부수효과가 있는 함수 안에는 빠져나오려고 하는 순수 함수가 있다"**

- original code

In [1]:
case class Player(name: String, score: Int)

def contest(p1: Player, p2: Player): Unit = 
    if(p1.score > p2.score){
        println(s"${p1.name} is the winner!")
    }else if(p1.score < p2.score){
        println(s"${p2.name} is the winner!")
    }else{
        println("It's a draw.")
    }

defined class Player
defined function contest

- code seperation

In [3]:
def winner(p1: Player, p2: Player): Option[Player] =
    if(p1.score > p2.score){
        Some(p1)
    }else if(p1.score < p2.score){
        Some(p2)
    }else{
        None
    }   

def contest(p1: Player, p2: Player): Unit = winner(p1, p2) match {
    case Some(Player(name, _)) => println(s"$name is the winner!")
    case None => println("It's a draw.")
}

defined function winner
defined function contest

- 순수 함수 : 서술(description)
- 불순 함수 : 해석기(interpreter)

#### 코드의 분리를 진행할 수록 부수효과들은 점점 바깥 계층으로 밀려나가며 순수한 core 부분이 많아진다

### require : Monad

In [1]:
trait Functor[F[_]] {
  def map[A,B](a: F[A])(f: A => B): F[B]
}

defined trait Functor

In [2]:
trait Monad[F[_]] extends Functor[F] {
    
  def unit[A](a: => A): F[A]
    
  def flatMap[A,B](a: F[A])(f: A => F[B]): F[B]

  def map[A,B](a: F[A])(f: A => B): F[B] 
    = flatMap(a)(a => unit(f(a)))
    
  def map2[A,B,C](a: F[A], b: F[B])(f: (A,B) => C): F[C] 
    = flatMap(a)(a => map(b)(b => f(a,b)))

  def replicateM[A](n: Int)(f: F[A]): F[List[A]] 
    = Stream.fill(n)(f).foldRight(unit(List[A]()))(map2(_,_)(_ :: _))
}

defined trait Monad

## 2. 소박한 IO monad : 그러나 문제점이 있는...

In [3]:
sealed trait IO[A] { self =>
                    
    def run: A
                    
    def map[B](f: A => B): IO[B]
    = new IO[B] { def run = f(self.run) }
                    
    def flatMap[B](f: A => IO[B]): IO[B]
    = new IO[B] { def run = f(self.run).run }
}

object IO extends Monad[IO] {
      
    def unit[A](a: => A): IO[A] = new IO[A] { def run = a }
    def flatMap[A,B](fa: IO[A])(f: A => IO[B]) = fa flatMap f
    def apply[A](a: => A): IO[A] = unit(a) // syntax for IO { ... }
}

def ReadLine: IO[String] 
= IO { readLine }

def PrintLine(msg: String): IO[Unit] 
= IO { println(msg) }

def fahrenheitToCelsius(f: Double): Double 
= (f - 32) * 5.0/9.0
  
def converter: IO[Unit] 
= for {
    _ <- PrintLine("Enter a temperature in degrees Fahrenheit: ")
    d <- ReadLine.map(_.toDouble)
    _ <- PrintLine(fahrenheitToCelsius(d).toString)
} yield ()

defined trait IO
defined object IO
defined function ReadLine
defined function PrintLine
defined function fahrenheitToCelsius
defined function converter

- StackOverflow : 스택넘침
- 과도한 일반화 : IO 라는 모나드 하나로 퉁치기 끝내기에는 입출력의 종류가 다양하다. 세분화가 필요.
- 차단식 입출력 : 비차단, 비동기 입출력 구현이 필요하다

## 3. 해결책 :
- Trampoline : 스택넘침 해결
- Free monad : 입출력 형태의 세분화가 가능한 추상 => 비차단, 비동기 가능

## 4. Trampoline

- StackOverflow Example

In [4]:
val f = (x: Int) => x

val g = List.fill(100000)(f).foldLeft(f)(_ compose _)

g(42)

: 

- Trampoline

In [20]:
sealed trait TailRec[A] {
    
    def flatMap[B](f: A => TailRec[B]): TailRec[B] 
    = FlatMap(this, f)
    
    def map[B](f: A => B): TailRec[B] 
    = flatMap(f andThen (Return(_)))
    = FlatMap(this, Return(f(_)))
    = FlatMap(this, i => Return(f(i)))
}

case class Return[A](a: A) extends TailRec[A]
case class Suspend[A](resume: () => A) extends TailRec[A]
case class FlatMap[A,B](sub: TailRec[A], k: A => TailRec[B]) extends TailRec[B]

object TailRec extends Monad[TailRec] {
    
    def unit[A](a: => A): TailRec[A] 
    = Return(a)
    
    def flatMap[A,B](a: TailRec[A])(f: A => TailRec[B]): TailRec[B] 
    = a flatMap f
    = FlatMap(a, f)
    
    def suspend[A](a: => TailRec[A]) 
    = Suspend(() => ()).flatMap { _ => a }
    = FlatMap(Suspend(() => ()), _ => a)
}

def run[A](t: TailRec[A]): A = t match {
    
    case Return(a) => a
    
    case Suspend(r) => r()
    
    case FlatMap(x, f) => x match {
      case Return(a) => run(f(a))
      case Suspend(r) => run(f(r()))
      case FlatMap(y, g) => run(y flatMap (a => g(a) flatMap f))
    }
}

defined trait TailRec
defined class Return
defined class Suspend
defined class FlatMap
defined object TailRec
defined function run

run(Return(a)) == a      
> 내용물 꺼내기

run(Suspend(r)) == r()          
> 내용물(함수, Function0, thunk) 실행

case FlatMap(x, f) == ?         

> x가 Return 인 경우, 함수 f 를 적용 후 재귀

> x가 Suspend 인 경우, thunk 실행 후 함수 f 를 적용, 그 후 재귀...

> x가 FlatMap 인 경우... y flatMap (a => g(a) flatMap f) 후 재귀????

**y flatMap (a => g(a) flatMap f) 의 흐름을 추적해보자!**

1. run(t)
2. t == FlatMap(x, f) 
3. x == FlatMap(y, g)
4. run(y flatMap (a => g(a) flatMap f))
5. y flatMap (a => g(a) flatMap f) 를 t1 이라고 부르자
FlatMap(y , a => g(a) flatMap f)

6. run(t1)
7. t1 == FlatMap(y, a => g(a) flatMap f)
8. y == FlatMap(z, h)
9. run(z flatMap (a => h(a) flatMap g))
10. z flatMap (a => h(a) flatMap g) 를 t2 라고 부르자

11. run(t2)
.
.
.